# Đề thi mẫu chứng chỉ TensorFlow - Dạng bài 03

```
ProtonX - TensorFlow Class 
```
Giới thiệu về chứng chỉ - https://www.tensorflow.org/certificate

> Chú ý: đọc kỹ các nội dung trong Handle Notebook trong liên kết trên.

---

### Hướng dẫn làm bài 

Đọc kỹ File `Tensorflow Exam - Tips` trên hệ thống Học tập.

*Một số yêu cầu:
- Sử dụng TensorFlow 2.x trở lên.

Phân loại ảnh thực tế:
- Một số loại dataset thường gặp trong đề thi: Chó - mèo, Ngựa - người, Dao - Búa - Kéo.
- Có thể đề yêu cầu load data từ tf.dataset. Đề bài có thể yêu cầu resize ảnh về kích thước nào đó (ví dụ 224x224x3) và re-scale input về khoảng [-1, 1]. Thực hiện resize và rescale ảnh trong hàm map:
https://www.tensorflow.org/api_docs/python/tf/data/Dataset#map

- Lưu ý 1: Có thể sử dụng Transfer learning - ưu tiên MobileNet để Train mô hình có độ chính xác cao hơn.
- Lưu ý 2: Phân loại nhị phân nhưng đề bài có thể yêu cầu lớp cuối cùng của model phải sử dụng hàm softmax(thay vì sigmoid), vì vậy cần chọn làm loss phù hợp với hàm softmax.
- flow_from_directory + generator
  + Chó mèo: https://youtu.be/kShvYHDXvvg
  + Ngựa người: https://youtu.be/ee9tF9xEf04
  + Dao búa kéo: https://www.tensorflow.org/datasets/catalog/rock_paper_scissors
  + Cách resize ảnh, rescale input và transfer learning có thể tham khảo: https://www.tensorflow.org/tutorials/images/transfer_learning
https://youtu.be/Y-4KLFt_c6Y

- Bài sử dụng transfer learning có thể có kết quả khi submit là 2/5, 3/5, …. Vì vậy nên submit nhiều lần. Nếu đạt từ 4/5 trở lên thì có thể làm bài khác (áp dụng với tất cả các bài)

- Chạy lại toàn bộ File này trước khi ấn nộp bài.

### Đề thi mẫu bài 03

In [1]:
# TensorFlow and tf.keras
import tensorflow as tf
import tensorflow_datasets as tfds
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# Helper libraries
import numpy as np
import matplotlib.pyplot as plt

print(tf.__version__)

2023-10-07 23:05:32.509840: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: SSE4.1 SSE4.2 AVX AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


ModuleNotFoundError: No module named 'tensorflow_datasets'

In [ ]:
# Các bạn có thể tải dữ liệu bằng đường liên kết này 
!wget https://storage.googleapis.com/download.tensorflow.org/data/rps.zip -O /tmp/rockpaperscissors.zip
!unzip /tmp/rockpaperscissors.zip

/bin/bash: /home/danieldu/anaconda3/lib/libtinfo.so.6: no version information available (required by /bin/bash)
wget: /home/danieldu/anaconda3/lib/libuuid.so.1: no version information available (required by wget)
--2023-10-07 22:56:35--  https://storage.googleapis.com/download.tensorflow.org/data/rps.zip
Resolving storage.googleapis.com (storage.googleapis.com)... 142.251.43.27, 142.251.42.251, 172.217.163.59, ...
Connecting to storage.googleapis.com (storage.googleapis.com)|142.251.43.27|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 200682221 (191M) [application/zip]
Saving to: ‘/tmp/rockpaperscissors.zip’

/tmp/rockpapersciss 100%[===================>] 191.38M  1.10MB/s    in 2m 54s  

2023-10-07 22:59:29 (1.10 MB/s) - ‘/tmp/rockpaperscissors.zip’ saved [200682221/200682221]



In [ ]:
# Hoặc tải dữ liệu trong TensorFlow Dataset
DATASET_NAME = 'rock_paper_scissors'

(dataset_train_raw, dataset_test_raw), dataset_info = tfds.load(
    name=DATASET_NAME,
    data_dir='tmp',
    with_info=True,
    as_supervised=True,
    split=[tfds.Split.TRAIN, tfds.Split.TEST],
)

In [ ]:
# Write your code here 
training_datagen = ImageDataGenerator(
      rescale = 1./255,
	    rotation_range=40,
      width_shift_range=0.2,
      height_shift_range=0.2,
      shear_range=0.2,
      zoom_range=0.2,
      horizontal_flip=True,
      fill_mode='nearest')

validation_datagen = ImageDataGenerator(rescale = 1./255)

train_generator = training_datagen.flow_from_directory(
	TRAINING_DIR,
	target_size=(150,150),
	class_mode='categorical',
  batch_size=126
)

validation_generator = validation_datagen.flow_from_directory(
	VALIDATION_DIR,
	target_size=(150,150),
	class_mode='categorical',
  batch_size=126
)


NameError: name 'TRAINING_DIR' is not defined

In [ ]:
model = tf.keras.models.Sequential([
    tf.keras.layers.Conv2D(64, (3,3), activation='relu', input_shape=(150, 150, 3)),
    tf.keras.layers.MaxPooling2D(2, 2),
    tf.keras.layers.Conv2D(64, (3,3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2,2),
    tf.keras.layers.Conv2D(128, (3,3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2,2),
    tf.keras.layers.Conv2D(128, (3,3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2,2),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dropout(0.5),
    tf.keras.layers.Dense(512, activation='relu'),
    tf.keras.layers.Dense(3, activation='softmax')
    ])

SyntaxError: invalid syntax (3961098768.py, line 1)

In [ ]:
# Train model 
model.summary()
model.compile(loss = 'categorical_crossentropy', optimizer='rmsprop', metrics=['accuracy'])
history = model.fit(train_generator, epochs=25, steps_per_epoch=20, validation_data = validation_generator, verbose =1)


In [ ]:
# Lưu Model và nộp 
model.save("mymodel.h5")